In [ ]:
# GPU/CPU variant: try installing jax (may work on GPU on Colab)
!pip -q install --upgrade pip
!pip -q install jax jaxlib  # may install CPU-only; if GPU wheel needed, see JAX install docs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.5 MB/s eta 0:00:00


In [ ]:
import jax, jax.numpy as jnp, time
print("JAX version:", jax.__version__)
print("Devices:", jax.devices())


JAX version: 0.7.2
Devices: [CudaDevice(id=0)]


In [ ]:
import numpy as np
key = jax.random.PRNGKey(0)

def init_params(in_dim, out_dim, key):
    return {
        'w': jax.random.normal(key, (in_dim, out_dim)) * 0.02,
        'b': jnp.zeros((out_dim,))
    }

def forward(params, x):
    return jnp.dot(x, params['w']) + params['b']

@jax.jit
def step(params, x, y):
    def loss_fn(p):
        pred = forward(p, x)
        return jnp.mean((pred - y)**2)
    loss, grads = jax.value_and_grad(loss_fn)(params)
    return loss, grads

# Data
x = jax.random.normal(key, (128, 128))
y = jax.random.normal(key, (128, 10))
params = init_params(128, 10, key)

# Warmup & measure
_ = step(params, x, y)  # compile
t0 = time.time()
for _ in range(50):
    l,g = step(params, x, y)
t1 = time.time()
print("JIT time per step (ms):", (t1-t0)/50*1000)


JIT time per step (ms): 0.2042865753173828


above is project4